# Uniswap V3 Concentrated Liquidity Backtesting Demo

This notebook demonstrates vectorized backtesting of Uniswap V3 concentrated liquidity positions.

Key concepts:
- **Tick ranges**: Liquidity is concentrated between tick_lower and tick_upper
- **In-range fees**: Only earn fees when price is in your range
- **Capital efficiency**: Tighter ranges = higher capital efficiency but more rebalancing
- **Impermanent loss**: Amplified in concentrated positions

In [ ]:
import sys
sys.path.append('..')

import ammbt as amm
import numpy as np
import pandas as pd
from ammbt.utils import generate_tick_ranges, create_v3_strategy_grid, tick_to_price
from ammbt.plotting import plot_performance, plot_efficient_frontier, plot_pnl_distribution
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

## 1. Generate Synthetic Swap Data

Same price movements as v2, but now we'll test concentrated positions.

In [ ]:
# Generate swaps with volatility
swaps = amm.generate_swaps(
    n_swaps=5000,
    initial_price=1.0,
    volatility=0.05,       # 5% volatility
    drift=0.001,           # Slight upward trend
    base_volume=100000,
    model='jump',          # Add price jumps
    seed=42,
)

print(f"Generated {len(swaps)} swaps")
print(f"Price range: {swaps['price'].min():.4f} - {swaps['price'].max():.4f}")
print(f"Price change: {(swaps['price'].iloc[-1] / swaps['price'].iloc[0] - 1) * 100:.2f}%")

# Plot price
fig = go.Figure()
fig.add_trace(go.Scatter(y=swaps['price'], mode='lines', name='Price'))
fig.update_layout(
    title='Price Path with Jumps',
    xaxis_title='Swap Index',
    yaxis_title='Price',
    template='plotly_dark',
    height=400,
)
fig.show()

## 2. Generate V3 Tick Ranges

Create concentrated liquidity positions with different range widths.

In [ ]:
# Generate tick ranges around current price
tick_ranges = generate_tick_ranges(
    current_price=1.0,
    range_widths_pct=[0.05, 0.10, 0.20, 0.50],  # ±5%, ±10%, ±20%, ±50%
    tick_spacings=[60],  # 0.3% fee tier
    num_ranges=4,
)

print(f"Generated {len(tick_ranges)} tick ranges:\n")
for i, (lower, upper) in enumerate(tick_ranges):
    lower_price = tick_to_price(lower)
    upper_price = tick_to_price(upper)
    width_pct = (upper_price - lower_price) / 1.0 * 100
    print(f"Range {i+1}: [{lower_price:.4f}, {upper_price:.4f}] (±{width_pct/2:.1f}%)")

## 3. Create Strategy Grid

Test combinations of:
- Capital amounts
- Tick ranges (narrow to wide)
- Rebalancing strategies

In [ ]:
# Create strategy grid
strategies = create_v3_strategy_grid(
    initial_capitals=[10_000, 50_000],
    tick_ranges=tick_ranges,
    rebalance_thresholds=[0.0],  # No price-based rebalancing
    rebalance_frequencies=[0, 100, 500],  # Never, frequent, occasional
)

print(f"Testing {len(strategies)} strategy variants\n")
print("Strategy grid:")
strategies.head(10)

## 4. Run V3 Backtest

Simulate all concentrated liquidity strategies.

In [ ]:
%%time

# Initialize V3 backtester
backtester = amm.LPBacktester(
    amm_type='v3',
    initial_price=1.0,
    initial_liquidity=5_000_000,
    fee_tier=3000,  # 0.3%
)

# Run simulation
results = backtester.run(swaps, strategies)

print(results)

## 5. Analyze Results

Compare concentrated vs. wide ranges.

In [ ]:
# Top strategies
top_10 = results.summary().head(10)

print("Top 10 Strategies by Net PnL:")
print("=" * 100)
top_10[[
    'net_pnl',
    'total_return_pct',
    'il_pct',
    'total_fees',
    'pct_time_in_range',
    'num_rebalances',
]]

In [ ]:
# Summary statistics
print("V3 Performance Statistics:")
print("=" * 100)
print(f"Mean Net PnL: ${results.metrics['net_pnl'].mean():,.2f}")
print(f"Best Net PnL: ${results.metrics['net_pnl'].max():,.2f}")
print(f"Worst Net PnL: ${results.metrics['net_pnl'].min():,.2f}")
print(f"\nMean IL: {results.metrics['il_pct'].mean():.2f}%")
print(f"Mean Fees: ${results.metrics['total_fees'].mean():,.2f}")
print(f"Mean Time in Range: {results.metrics['pct_time_in_range'].mean():.1f}%")
print(f"\nStrategies with positive PnL: {(results.metrics['net_pnl'] > 0).sum()} / {len(results.metrics)}")

## 6. Range Width Analysis

How does range width affect performance?

In [ ]:
# Add range width to results
combined = pd.concat([strategies, results.metrics], axis=1)

# Calculate range width in price terms
combined['range_lower_price'] = combined['tick_lower'].apply(tick_to_price)
combined['range_upper_price'] = combined['tick_upper'].apply(tick_to_price)
combined['range_width_pct'] = (
    (combined['range_upper_price'] - combined['range_lower_price']) / 1.0 * 100
)

# Group by range width
range_analysis = combined.groupby('range_width_pct').agg({
    'net_pnl': 'mean',
    'total_fees': 'mean',
    'il_pct': 'mean',
    'pct_time_in_range': 'mean',
    'num_rebalances': 'mean',
}).round(2)

print("Performance by Range Width:")
print("=" * 100)
range_analysis

In [ ]:
# Plot range width vs performance
fig = go.Figure()

# Scatter plot
for capital in combined['initial_capital'].unique():
    subset = combined[combined['initial_capital'] == capital]
    fig.add_trace(go.Scatter(
        x=subset['range_width_pct'],
        y=subset['net_pnl'],
        mode='markers',
        name=f'${int(capital):,}',
        marker=dict(size=8),
    ))

fig.update_layout(
    title='Range Width vs. Net PnL',
    xaxis_title='Range Width (%)',
    yaxis_title='Net PnL ($)',
    template='plotly_dark',
    height=500,
)
fig.show()

## 7. Visualize Best Strategy

Detailed view of top-performing concentrated position.

In [ ]:
# Best strategy
best_idx = results.metrics['net_pnl'].idxmax()

print(f"Best Strategy (Index {best_idx}):")
print("=" * 100)
best_strategy = combined.loc[best_idx]
print(f"Capital: ${best_strategy['initial_capital']:,.0f}")
print(f"Range: [{best_strategy['range_lower_price']:.4f}, {best_strategy['range_upper_price']:.4f}]")
print(f"Range Width: ±{best_strategy['range_width_pct']/2:.1f}%")
print(f"\nNet PnL: ${best_strategy['net_pnl']:,.2f}")
print(f"Total Return: {best_strategy['total_return_pct']:.2f}%")
print(f"IL: {best_strategy['il_pct']:.2f}%")
print(f"Fees: ${best_strategy['total_fees']:,.2f}")
print(f"Time in Range: {best_strategy['pct_time_in_range']:.1f}%")
print(f"Rebalances: {int(best_strategy['num_rebalances'])}")

In [ ]:
# Plot performance
fig = plot_performance(results, strategy_idx=best_idx, show_reserves=False)
fig.show()

## 8. Capital Efficiency Analysis

V3's key value proposition: earn more fees with less capital.

In [ ]:
# Calculate fee yield (fees / capital)
combined['fee_yield_pct'] = (combined['total_fees'] / combined['initial_capital']) * 100

# Plot fee yield vs. time in range
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=combined['pct_time_in_range'],
    y=combined['fee_yield_pct'],
    mode='markers',
    marker=dict(
        size=8,
        color=combined['range_width_pct'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Range Width (%)'),
    ),
    text=combined['range_width_pct'].round(1),
    hovertemplate=(
        'Time in Range: %{x:.1f}%<br>'
        'Fee Yield: %{y:.2f}%<br>'
        'Range Width: %{text}%<br>'
        '<extra></extra>'
    ),
))

fig.update_layout(
    title='Capital Efficiency: Fee Yield vs. Time in Range',
    xaxis_title='Time in Range (%)',
    yaxis_title='Fee Yield (%)',
    template='plotly_dark',
    height=500,
)
fig.show()

## 9. Rebalancing Impact

Compare no rebalancing vs. active management.

In [ ]:
# Group by rebalancing strategy
no_rebalance = combined[combined['rebalance_frequency'] == 0]
frequent_rebalance = combined[combined['rebalance_frequency'] == 100]
occasional_rebalance = combined[combined['rebalance_frequency'] == 500]

comparison = pd.DataFrame({
    'Strategy': ['No Rebalancing', 'Frequent (100)', 'Occasional (500)'],
    'Mean PnL': [
        no_rebalance['net_pnl'].mean(),
        frequent_rebalance['net_pnl'].mean(),
        occasional_rebalance['net_pnl'].mean(),
    ],
    'Mean Fees': [
        no_rebalance['total_fees'].mean(),
        frequent_rebalance['total_fees'].mean(),
        occasional_rebalance['total_fees'].mean(),
    ],
    'Mean Time in Range': [
        no_rebalance['pct_time_in_range'].mean(),
        frequent_rebalance['pct_time_in_range'].mean(),
        occasional_rebalance['pct_time_in_range'].mean(),
    ],
    'Avg Rebalances': [
        no_rebalance['num_rebalances'].mean(),
        frequent_rebalance['num_rebalances'].mean(),
        occasional_rebalance['num_rebalances'].mean(),
    ],
    'Avg Gas Costs': [
        no_rebalance['gas_costs'].mean(),
        frequent_rebalance['gas_costs'].mean(),
        occasional_rebalance['gas_costs'].mean(),
    ],
})

print("\nRebalancing Strategy Comparison:")
print("=" * 100)
comparison

## 10. Key Insights: V3 vs. V2

What did we learn about concentrated liquidity?

In [ ]:
print("\nKEY INSIGHTS FROM V3 BACKTESTING")
print("=" * 100)
print("\n1. RANGE WIDTH TRADEOFF:")
print("   - Narrow ranges: Higher fee yield when in range, but more time out of range")
print("   - Wide ranges: More consistent fee earnings, but lower capital efficiency")
print("\n2. REBALANCING:")
print("   - Essential for narrow ranges in volatile markets")
print("   - Costs must be weighed against lost fee opportunities")
print("\n3. IMPERMANENT LOSS:")
print("   - Amplified in concentrated positions")
print("   - Fees must compensate for increased IL")
print("\n4. OPTIMAL STRATEGY:")
print("   - Depends on expected volatility and swap volume")
print("   - High volume + low volatility = narrow ranges win")
print("   - Low volume + high volatility = wider ranges win")

## Conclusion

Uniswap V3 concentrated liquidity enables:
- **Higher capital efficiency** through focused ranges
- **Strategy customization** via tick bounds
- **Active management** opportunities through rebalancing

But requires:
- **Careful range selection** based on volatility
- **Active monitoring** to stay in range
- **Gas cost management** for rebalancing

Next: Test with real historical data to validate strategies!